In [7]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from pathlib import Path

# 1) 파일 경로
paths = {
    "mapo":      "data/마포도로망/마포도로망_test.csv",
    "seodaemun": "data/서대문도로망/서대문도로망_test.csv",
    "eunpyeong": "data/은평도로망/은평도로망_test.csv",
}

# 2) 컬럼 순서(최종 출력 순서). 없으면 자동 생성되어 NaN 채움
ORDER = [
    "u", "v", "highway", "oneway", "length", "bridge", "tunnel",
    "avg_height", "avg_slope", "lanes", "width", "up_lanes", "down_lanes", "surface" # 소스 구분 컬럼 추가(유지/분석용)
]

# ---- 헬퍼: 불리언/yes/no → 1/0 통일 ----------------------------------------
def normalize_boolish(s: pd.Series) -> pd.Series:
    return (
        s.replace({
            True: 1, False: 0,
            "True": 1, "False": 0,
            "true": 1, "false": 0,
            "YES": 1, "NO": 0,
            "Yes": 1, "No": 0,
            "Y": 1, "N": 0,
            "yes": 1, "no": 0,
            1.0: 1, 0.0: 0
        })
    )

# ---- 헬퍼: 정수 유사값만 Int64로 캐스팅(소수 있으면 NaN) ---------------------
def cast_nullable_int(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    # 정수처럼 보이는 값만 통과 (NaN은 그대로 허용)
    is_intlike = s.isna() | np.isclose(s, np.round(s))
    s = s.where(is_intlike, np.nan)
    return s.round().astype("Int64")

# ---- 헬퍼: Float64 캐스팅 ----------------------------------------------------
def cast_float(s: pd.Series) -> pd.Series:
    return pd.to_numeric(s, errors="coerce").astype("Float64")

# ---- 파일 로드 + 정렬(타입/컬럼 체계화) --------------------------------------
def load_and_align(fp: str, source: str) -> pd.DataFrame:
    df = pd.read_csv(fp)
    df["source"] = source

    # 1) 불리언 유사 컬럼 정규화(있을 때만)
    for c in ["oneway", "bridge", "tunnel"]:
        if c in df.columns:
            df[c] = normalize_boolish(df[c])

    # 2) 정수 컬럼(정수 유사만 Int64로)
    for c in ["oneway", "bridge", "tunnel", "lanes", "width", "up_lanes", "down_lanes"]:
        if c in df.columns:
            df[c] = cast_nullable_int(df[c])

    # 3) 실수 컬럼(Float64)
    for c in ["length", "avg_height", "avg_slope"]:
        if c in df.columns:
            df[c] = cast_float(df[c])

    # 4) u, v (OSM ID) — 정수 유사만 Int64로. 소수 섞이면 NaN 처리
    for c in ["u", "v"]:
        if c in df.columns:
            df[c] = cast_nullable_int(df[c])

    # 5) surface는 문자열로 통일(있으면)
    if "surface" in df.columns:
        df["surface"] = df["surface"].astype("string")

    # 6) 최종 ORDER 충족: 누락 컬럼은 생성(NaN)
    for col in ORDER:
        if col not in df.columns:
            df[col] = pd.NA

    # 7) 컬럼 순서 고정
    df = df[ORDER]

    return df

# 3) 로드 + 병합
dfs = [load_and_align(fp, name) for name, fp in paths.items()]
merged = pd.concat(dfs, ignore_index=True)

# 4) (옵션) 동일 엣지(u, v) 중복 제거 — 필요 시 주석 해제
# merged = (
#     merged.sort_values(["u","v","source"])
#           .drop_duplicates(subset=["u","v"], keep="first")
#           .reset_index(drop=True)
# )

# 5) 저장
out_path = Path("data/은마서_도로망/마포_서대문_은평_도로망_병합_v2.csv")
out_path.parent.mkdir(parents=True, exist_ok=True)
merged.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"Saved: {out_path} | rows={len(merged):,} | cols={len(merged.columns)}")

# 6) 간단 점검
print("\n[dtypes]")
print(merged.dtypes)
if {"u","v"} <= set(merged.columns):
    print("[check] (u,v) NaN rows:", int(merged[["u","v"]].isna().any(axis=1).sum()))


Saved: data\은마서_도로망\마포_서대문_은평_도로망_병합_v2.csv | rows=27,033 | cols=14

[dtypes]
u                      Int64
v                      Int64
highway               object
oneway                 Int64
length               Float64
bridge                 Int64
tunnel                 Int64
avg_height           Float64
avg_slope            Float64
lanes                  Int64
width                  Int64
up_lanes               Int64
down_lanes             Int64
surface       string[python]
dtype: object
[check] (u,v) NaN rows: 0
